In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df= pd.read_csv("/kaggle/input/playground-series-s5e4/train.csv")

In [ ]:
print(df.isna().any())

Data set columns, Missing Values 
* id
* Podcast_Name
* Episode_ Title
* **Episode_Length_minutes**,  ****87093****
* Genre
* Host_Popularity_percentage
* Publication_Day
* Publication_Time
* **Guest_Popularity_percentage**, ****146030****
* Number_of_Ads
* Episode_Sentiment
* Listening_Time_minutes 

In [ ]:
#actual values
Y_train = df["Listening_Time_minutes"]

#dropped with missing values
df_new = df.drop(["Episode_Length_minutes", "Guest_Popularity_percentage", "id", "Listening_Time_minutes"], axis=1)

In [ ]:
#use only episode number
df_new["Episode_Title"] = df_new["Episode_Title"].str.extract(r'Episode\s*(\d+)')

In [ ]:
#label encoding

df['Genre'] = df['Genre'].astype('category').cat.codes
df['Publication_Day'] = df['Publication_Day'].astype('category').cat.codes
df['Publication_Time'] = df['Publication_Time'].astype('category').cat.codes
df['Episode_Sentiment'] = df['Episode_Sentiment'].astype('category').cat.codes

# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# df_new["Genre"] = le.fit_transform(df_new["Genre"])

In [ ]:
print(df_new)

In [ ]:
#use w2vec later on but temp drop of podcast_name
df_new = df_new.drop(["Podcast_Name"], axis=1)
print(df_new)

In [ ]:
print(df_new.isna().any())

In [ ]:
#mean normlization
df_new["Host_Popularity_percentage"] = df_new["Host_Popularity_percentage"].astype(float)
df_new["Episode_Title"] = df_new["Episode_Title"].astype(float)

df_new["Host_Popularity_percentage"]=(df_new["Host_Popularity_percentage"]-df_new["Host_Popularity_percentage"].mean())/df_new["Host_Popularity_percentage"].std()
df_new["Episode_Title"]=(df_new["Episode_Title"]-df_new["Episode_Title"].mean())/df_new["Episode_Title"].std()

In [ ]:
#model building
import tensorflow as tf
import keras
from keras import layers

model = keras.Sequential()

#model.add(keras.Input(shape=df_new.shape))

model.add(layers.Dense(50, activation="relu", name="layer1"))
model.add(layers.Dense(30, activation="relu", name="layer2"))
model.add(layers.Dense(10, name="layer3"))

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.MeanSquaredError(),
    metrics=[
        keras.metrics.RootMeanSquaredError()
    ]
)

model.summary()

In [ ]:
# nan_indices = np.where(np.isnan(df_new.to_numpy()))
# print(nan_indices)

# nan_rows = df_new.iloc[nan_indices[0]] 
# print(nan_rows)

df_new["Number_of_Ads"] = df_new["Number_of_Ads"].ffill()

In [ ]:
print(df_new.isna().any())

In [ ]:
model.fit(df_new, Y_train, batch_size=32, epochs=10)